# Convert CSV files to Db2 Tables

## Introduction

This notebook contains code for converting the csv files into Db2 tables. The `sqlalchemy` package will create the engine that will connect to the Db2 server, where we will be reading and writing to. This is a common python package used with many database types including Db2, you can find the official documentation for using the package [here](https://docs.sqlalchemy.org). The credentials have been provided with the accelerator and can be found in the `Db2-Connection` with the data assets. We load in the credentials using the `project_lib` package and create the engine in the required format. We then use `pandas` to read in the csv file, converting the file to a `pandas` dataframe, and then turning the dataframe into a Db2 table on the Db2 server using the `sqlalchemy` engine. The following files are converted:

 - bramches.csv
 - loan.csv
 - client.csv
 - disp.csv
 - card.csv
 - trans.csv
 - account.csv
 
 

This project uses a connection to an existing Db2 server which contains the credentials for reading and writing to the `Loan Default` schema. If you are setting up a connection to a new Db2 service, you can find more information on how to set up a connection [here](https://www.ibm.com/support/producthub/icpdata/docs/content/SSQNUZ_current/wsj/manage-data/conn_types.html).

**Sample Materials, provided under license. <br>
Licensed Materials - Property of IBM. <br>
© Copyright IBM Corp. 2020. All Rights Reserved. <br>
US Government Users Restricted Rights - Use, duplication or disclosure restricted by GSA ADP Schedule Contract with IBM Corp.<br>**

## 0. Load Libraries 

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Reading in Project Credentials
from project_lib import Project

# Converting CSV to DB2 Tables
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR, CHAR, DECIMAL, INTEGER, DATE
from sqlalchemy.schema import CreateSchema

<br/> 

## 1. Create the DB2 Connection

This section uses the credentials included in the `Db2-Connection` to connect to the Db2 server. Here we use the `project_lib` package to get the credentials from the connection and create our `sqlalchemy` engine. 

### 1.0 Collect Credentials 

Through the `project_lib` package we can gather the credential information from the existing connection provided by the accelerator. This allows the use of the notebook to be able to use the credentials without


In [2]:
CONNECTION_NAME = "Db2-Connection"

project = Project()
credentials = project.get_connection(name=CONNECTION_NAME)

<br/> 

### 1.1 Create Connection Engine

The `sqlalchemy` engine is the package we will use to interact with the DB2 server. The url format is specific to the database type you are interacting with. In our case we use the information loaded into the credential object above to complete the url and connect to the server.

In [4]:
# Set to empty string '' if SSL is not enabled on your connection
sslConnection = ';SECURITY=SSL;'
url = f"ibm_db_sa://{credentials['username']}:{credentials['password']}@{credentials['host']}:{credentials['port']}/{credentials['database']}{sslConnection}"
engine = create_engine(url)
connection = engine.connect()

<br/> 

## 2. Read in Data & Save as a Table

### 2.0 BRANCHES Table

Read in data from csv, create table in `LAON DEFAULT` schema, read in data from DB2 table to check data was loaded successfully. The `pandas` data types don't always map to the column type we want in Db2 so we create a dictionary for each table that contains the correct mapping and pass it into the `to_sql()` method.

In [6]:
df = pd.read_csv('/project_data/data_asset/branches.csv')
branches_dtypes = {
    'branch_id': CHAR(9),
    'branch_city': VARCHAR(50),
    'branch_zipcode': CHAR(5),
    'branch_state': CHAR(2),
    'branch_address': VARCHAR(50)   
}


try:
    df.to_sql('branches', connection, 'LOAN_DEFAULT', if_exists='fail', index=False, dtype=branches_dtypes)
    print('Created New Table')
    display(pd.read_sql('select * from LOAN_DEFAULT.BRANCHES', connection).head())
except ValueError:
    print('Table Already exists')
    display(pd.read_sql('select * from LOAN_DEFAULT.BRANCHES', connection).head())

Table Already exists


branch_id branch_city branch_zipcode branch_state   branch_address
0  B00000001    New York          10018           NY   400 W 37th St 
1  B00000002    New York          10007           NY       25 Park Pl
2  B00000003      Albany          12207           NY    21 N Main Ave
3  B00000004     Yonkers          10701           NY   270 N Broadway
4  B00000005   Rochester          14614           NY  14 S Hanover St

<br/> 

### 2.1 LOAN Table

In [8]:
df = pd.read_csv('/project_data/data_asset/loan.csv')
loan_dtypes = {
    'loan_id': CHAR(9),
    'account_id': CHAR(9),
    'loan_principal': INTEGER, 
    'loan_duration': INTEGER,
    'loan_payments': INTEGER,
    'loan_start_date': DATE,
    'loan_purpose': VARCHAR(50)
}

try:
    df.to_sql('loan', connection, 'LOAN_DEFAULT', if_exists='fail', index=False, dtype=loan_dtypes)
    print('Created New Table')
    display(pd.read_sql('select * from LOAN_DEFAULT.LOAN', connection).head())
except ValueError:
    print('Table Already exists')
    display(pd.read_sql('select * from LOAN_DEFAULT.LOAN', connection).head())

Table Already exists


loan_id account_id  loan_principal  loan_duration  loan_payments  \
0  L00005657  A00003354            4980             12            415   
1  L00006234  A00006061            5148             12            429   
2  L00006699  A00008330            7656             24            319   
3  L00006688  A00008268            8616             24            359   
4  L00006312  A00006453           10944             36            304   

  loan_start_date loan_purpose  
0      2014-07-05          car  
1      2018-05-28          car  
2      2014-03-30          car  
3      2017-09-26          car  
4      2018-05-17          car

<br/> 

### 2.2 CLIENT Table

In [9]:
df = pd.read_csv('/project_data/data_asset/client.csv', dtype={'zipcode':str})
client_dtypes = {
    'client_id': CHAR(9), 
    'sex': VARCHAR(50),
    'birth_date': DATE,
    'social': CHAR(11),
    'first': VARCHAR(50),
    'middle': VARCHAR(50),
    'last': VARCHAR(50),
    'phone': VARCHAR(50),
    'email': VARCHAR(50),
    'address_1': VARCHAR(50),
    'address_2': VARCHAR(50),
    'city' : VARCHAR(50),
    'state': CHAR(2),
    'zipcode': CHAR(5)
}

try:
    df.to_sql('client', connection, 'LOAN_DEFAULT', if_exists='fail', index=False, dtype=client_dtypes)
    print('Created New Table')
    display(pd.read_sql('select * from LOAN_DEFAULT.CLIENT', connection).head())
except ValueError:
    print('Table Already exists')
    display(pd.read_sql('select * from LOAN_DEFAULT.CLIENT', connection).head())

Table Already exists


client_id     sex  birth_date       social   first      middle      last  \
0  C00000001  Female  1990-12-13  926-93-2157    Emma       Avaya     Smith   
1  C00000002    Male  1965-02-04  806-94-5725    Noah     Everest  Thompson   
2  C00000003  Female  1960-10-09  614-70-9100  Olivia  Brooklynne   Johnson   
3  C00000004    Male  1976-12-01  580-20-3414    Liam       Irvin     White   
4  C00000005  Female  1980-07-03  536-14-5809  Sophia       Danae  Williams   

          phone                       email            address_1 address_2  \
0  367-171-6840        emma.smith@gmail.com  75 W. Berkshire St.  Apt. 705   
1  212-423-7734     noah.thompson@gmail.com  75 W. Berkshire St.  Apt. 102   
2  212-425-6932  olivia.johnson@outlook.com  75 W. Berkshire St.  Apt. 102   
3  951-567-8925        liam.white@gmail.com   708 Oak Valley St.      None   
4  428-265-1568   sophia.williams@gmail.com   708 Oak Valley St.      None   

            city state zipcode  
0  New York City    NY   10040  
1  New York City    NY   10040  
2  New York City    NY   10040  
3        Yonkers    NY   10701  
4        Yonkers    NY   10701

<br/> 

### 2.3 DISP Table

In [10]:
df = pd.read_csv('/project_data/data_asset/disp.csv')
disp_dtypes = {
    'disp_id': VARCHAR(50),
    'account_id': VARCHAR(50),
    'type': VARCHAR(50),
    'client_id': VARCHAR(50)
}

try:
    df.to_sql('disp', connection, 'LOAN_DEFAULT', if_exists='fail', index=False, dtype=disp_dtypes)
    print('Created New Table')
    display(pd.read_sql('select * from LOAN_DEFAULT.DISP', connection).head())
except ValueError:
    print('Table Already exists')
    display(pd.read_sql('select * from LOAN_DEFAULT.DISP', connection).head())

Table Already exists


disp_id account_id   type  client_id
0  D00000001  A00000001  Owner  C00000001
1  D00000002  A00000002  Owner  C00000002
2  D00000003  A00000002   User  C00000003
3  D00000004  A00000003  Owner  C00000004
4  D00000005  A00000003   User  C00000005

<br/> 

### 2.4 CARD Table

In [11]:
df = pd.read_csv('/project_data/data_asset/card.csv')
card_dtypes = {
    'card_id': CHAR(9),
    'disp_id': CHAR(9),
    'debit_card_type': VARCHAR(50),
    'issued_date': DATE
}

try:
    df.to_sql('card', connection, 'LOAN_DEFAULT', if_exists='fail', index=False, dtype=card_dtypes)
    print('Created New Table')
    display(pd.read_sql('select * from LOAN_DEFAULT.CARD', connection).head())
except ValueError:
    print('Table Already exists')
    display(pd.read_sql('select * from LOAN_DEFAULT.CARD', connection).head())

Table Already exists


card_id    disp_id debit_card_type issued_date
0  V00001005  D00009285  VISA Signature  2013-11-07
1  V00000104  D00000588  VISA Signature  2014-01-19
2  V00000747  D00004915  VISA Signature  2014-02-05
3  V00000070  D00000439  VISA Signature  2014-02-08
4  V00000577  D00003687  VISA Signature  2014-02-15

<br/> 

### 2.5 TRANS Table

In [12]:
df = pd.read_csv('/project_data/data_asset/trans.csv')
trans_dtypes = {
    'trans_id': CHAR(9),
    'account_id': CHAR(9),
    'transaction_date': DATE,
    'tx_type_1': VARCHAR(50),
    'tx_type_2': VARCHAR(50),
    'tx_type_3': VARCHAR(50),
    'amount': DECIMAL
}

try:
    df.to_sql('trans', connection, 'LOAN_DEFAULT', if_exists='fail', index=False, dtype=trans_dtypes)
    print('Created New Table')
    display(pd.read_sql('select * from LOAN_DEFAULT.TRANS', connection).head())
except ValueError:
    print('Table Already exists')
    display(pd.read_sql('select * from LOAN_DEFAULT.TRANS', connection).head())

Table Already exists


trans_id account_id transaction_date tx_type_1       tx_type_2 tx_type_3  \
0  T00695247  A00002378       2013-01-01    Credit  Credit in Cash      None   
1  T00171812  A00000576       2013-01-01    Credit  Credit in Cash      None   
2  T00207264  A00000704       2013-01-01    Credit  Credit in Cash      None   
3  T01117247  A00003818       2013-01-01    Credit  Credit in Cash      None   
4  T00579373  A00001972       2013-01-02    Credit  Credit in Cash      None   

   amount  
0   700.0  
1   900.0  
2  1000.0  
3   600.0  
4   400.0

<br/> 

### 2.6 ACCOUNT Table

In [13]:
df = pd.read_csv('/project_data/data_asset/account.csv')
account_dtypes = {
    'account_id': CHAR(9),
    'frequency': VARCHAR(50),
    'creation_date': DATE,
    'branch_id': CHAR(9)
}


try:
    df.to_sql('account', connection, 'LOAN_DEFAULT', if_exists='fail', index=False, dtype=account_dtypes)
    print('Created New Table')
    display(pd.read_sql('select * from LOAN_DEFAULT.ACCOUNT', connection).head())
except ValueError:
    print('Table Already exists')
    display(pd.read_sql('select * from LOAN_DEFAULT.ACCOUNT', connection).head())

Table Already exists


account_id         frequency creation_date  branch_id
0  A00000576  Monthly Issuance    2013-01-01  B00000006
1  A00003818  Monthly Issuance    2013-01-01  B00000006
2  A00000704  Monthly Issuance    2013-01-01  B00000006
3  A00002378  Monthly Issuance    2013-01-01  B00000006
4  A00002632  Monthly Issuance    2013-01-02  B00000006

<br/> 

### 2.7 BALANCE Table

In [14]:
df = pd.read_csv('/project_data/data_asset/balance.csv')
balance_dtypes = {
    'account_id': CHAR(9),
    'transaction_period': CHAR(7),
    'balance_mean': DECIMAL,
    'balance_std': DECIMAL,
    'balance_last': DECIMAL
}

try:
    df.to_sql('balance', connection, 'LOAN_DEFAULT', if_exists='fail', index=False, dtype=balance_dtypes)
    print('Created New Table')
    display(pd.read_sql('select * from LOAN_DEFAULT.BALANCE', connection).head())
except ValueError:
    print('Table Already exists')
    display(pd.read_sql('select * from LOAN_DEFAULT.BALANCE', connection).head())

Table Already exists


account_id transaction_period  balance_mean  balance_std  balance_last
0  A00000001            2015-03        1000.0          NaN        1000.0
1  A00000001            2015-04       13085.0       7280.0       17298.0
2  A00000001            2015-05       22403.0       1235.0       23156.0
3  A00000001            2015-06       26735.0        100.0       26735.0
4  A00000001            2015-07       26917.0       3029.0       25223.0